# Correlation analysis for Majority rule for group size of 5.
Date: 26.11.2025

In this notebook I do the correlation analyisis to look for what internal variable correlates the best with the chi-values.

Steps:
1. Normalize the data
2. Compute Spearman, Mutual Information, and Random Forest Importance
3. Analyze micro and macro variables
4. Produce comparision charts

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde


import sys
import os

In [2]:
def kde_mi(x, y, gridsize=100, bw_method=None):
    """
    KDE-based mutual information matching ISOKANN.jl implementation
    """
    # Create grid
    xg = np.linspace(x.min(), x.max(), gridsize)
    yg = np.linspace(y.min(), y.max(), gridsize)
    dx = xg[1] - xg[0]
    dy = yg[1] - yg[0]
    
    # Estimate joint density
    xy = np.vstack([x, y])
    kde_joint = gaussian_kde(xy, bw_method=bw_method)
    xg_mesh, yg_mesh = np.meshgrid(xg, yg)
    positions = np.vstack([xg_mesh.ravel(), yg_mesh.ravel()])
    pxy = kde_joint(positions).reshape(gridsize, gridsize).T
    
    # Estimate marginals
    px = np.sum(pxy, axis=1, keepdims=True) * dy
    py = np.sum(pxy, axis=0, keepdims=True) * dx
    
    # Compute MI (avoid log(0))
    px_py = px @ py
    with np.errstate(divide='ignore', invalid='ignore'):
        integrand = pxy * np.log(pxy / px_py)
        integrand[~np.isfinite(integrand)] = 0
    
    return np.sum(integrand) * dx * dy

In [ ]:
#1 Load data
# Get the absolute path of the parent directory (project root)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..','..','..'))

# Add it to the Python path
if project_root not in sys.path:
    sys.path.append(project_root)

sims_path = os.path.join('ABa-KiTo', 'majority_rule', 'data', 'simulations','')
data_dir =  os.path.join(project_root, sims_path)

#Load data formatted for ISOKANN.jl from ABM simulations
# Read data
states_data = np.load(data_dir + '2025-11-26-dataCorrelation_majority_rule.npz')
xs = states_data['xs'] #xs.shape (n_dim, n_samples)


# # # --- Load chi function ---
chivals_path = os.path.join('ABa-KiTo', 'majority_rule','data',  'chi_vals', '')
chivals_dir = os.path.join(project_root, chivals_path)
chi0 = np.load(chivals_dir + 'chi_values_MR.npz')

print("data loaded!")

#load data with names of macro vars
macro_vars = np.load(data_dir + '2025-11-26_macro_vars_names.npz')

data loaded!


In [4]:
# 2 Normalize the microvariables
n_micro = 200
n_macro = 12

xs_micro = xs[:n_micro,:]
xs_macro = xs[-n_macro:, :]

scaler = StandardScaler()
xs_micro_scaled = scaler.fit_transform(xs_micro)

In [5]:
# Combine micro + macro into single DataFrame for analysis
all_features = np.vstack((xs_micro_scaled, xs_macro))
#ll_features = xs
#labels = [f"agent_{i}" for i in range(n_micro)] + [f"macro_{i}" for i in range(n_macro)]
labels = []
for i in range(100):
    n = i 
    labels.append(f"agent_{i} PB")
    labels.append(f"agent_{i} PG")

labels += [f"macro_{i}" for i in range(n_macro)]

df = pd.DataFrame(all_features.T, columns=labels)
df["chi"] = chi0

In [6]:
df.columns

Index(['agent_0 PB', 'agent_0 PG', 'agent_1 PB', 'agent_1 PG', 'agent_2 PB',
       'agent_2 PG', 'agent_3 PB', 'agent_3 PG', 'agent_4 PB', 'agent_4 PG',
       ...
       'macro_3', 'macro_4', 'macro_5', 'macro_6', 'macro_7', 'macro_8',
       'macro_9', 'macro_10', 'macro_11', 'chi'],
      dtype='object', length=213)

In [7]:
# =====================================================
# === 1. Spearman Correlation =========================
# =====================================================
print("Computing Spearman correlations...")
spearman_corr = []
for var in labels:
    rho, _ = spearmanr(df[var], df["chi"])
    spearman_corr.append(rho)

# =====================================================
# === 2. Mutual Information ===========================
# =====================================================
print("Computing Mutual Information...")
mi = mutual_info_regression(df[labels], df["chi"], random_state=42)

# =====================================================
# === 3.  KDE-based Mutual Information  ===============
# =====================================================
print("\nComputing KDE-based Mutual Information...")
feature_columns = [col for col in df.columns if col != "chi"]
mi_kde = [kde_mi(df["chi"].values, df[col].values) for col in feature_columns]


# # =====================================================
# # === 3. Random Forest Feature Importance =============
# # =====================================================
# print("Training Random Forest for feature importance...")
# rf = RandomForestRegressor(
#     n_estimators=100,
#     max_depth=10,
#     random_state=42,
#     n_jobs=-1
# )
# rf.fit(df[labels], df["chi"])
# rf_importance = rf.feature_importances_

# =====================================================
# === 4. Summarize Results ============================
# =====================================================
results = pd.DataFrame({
    "variable": labels,
    "spearman": spearman_corr,
    "mutual_info": mi,
    "mi_kde": mi_kde
})

results["abs_spearman"] = results["spearman"].abs()
results.sort_values("abs_spearman", ascending=False, inplace=True)

# Print top 10
print("\nTop 10 χ-correlated variables:")
print(results.head(10))

Computing Spearman correlations...
Computing Mutual Information...

Computing KDE-based Mutual Information...

Top 10 χ-correlated variables:
        variable  spearman  mutual_info    mi_kde  abs_spearman
211     macro_11  0.809455     1.669447  0.654837      0.809455
208      macro_8 -0.809455     1.668637  0.654837      0.809455
9     agent_4 PG  0.763240     1.932331  0.579513      0.763240
19    agent_9 PG  0.761051     1.925574  0.570507      0.761051
57   agent_28 PG  0.760965     1.927089  0.570107      0.760965
79   agent_39 PG  0.760811     1.929215  0.574712      0.760811
71   agent_35 PG  0.759625     1.925038  0.569438      0.759625
179  agent_89 PG  0.759582     1.924551  0.571845      0.759582
193  agent_96 PG  0.759169     1.922108  0.563672      0.759169
69   agent_34 PG  0.758203     1.919428  0.567936      0.758203


In [8]:
results["mi_kde_clipped"] = results["mi_kde"].clip(lower=0)

macro_vars = results[results["variable"].str.contains("macro")]


print(macro_vars[['variable', 'spearman', 'mi_kde_clipped']].style.format(precision=3).hide(axis='index').to_latex()
        )

\begin{tabular}{lrr}
variable & spearman & mi_kde_clipped \\
macro_11 & 0.809 & 0.655 \\
macro_8 & -0.809 & 0.655 \\
macro_9 & 0.693 & 0.531 \\
macro_10 & 0.673 & 0.532 \\
macro_6 & 0.344 & 0.000 \\
macro_0 & 0.344 & 0.000 \\
macro_4 & 0.314 & 0.000 \\
macro_7 & -0.192 & 0.140 \\
macro_3 & -0.191 & 0.140 \\
macro_5 & -0.185 & 0.140 \\
macro_1 & -0.178 & 0.140 \\
macro_2 & -0.171 & 0.140 \\
\end{tabular}



In [33]:
#load data with names of macro vars
macro_vars_name = np.load(data_dir + '2025-11-26_macro_vars_names.npz')

# List the keys stored inside the npz file
print(macro_vars_name.files)

macros_names =macro_vars_name['macro_vars_names']

['macro_vars_names']


In [34]:
lookup = {f"macro_{i}": name for i, name in enumerate(macros_names)}


In [40]:
macro_vars['real_name'] = macro_vars['variable'].map(lookup)
macro_vars

C:\Users\Flo\AppData\Local\Temp\ipykernel_9676\1528150948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macro_vars['real_name'] = macro_vars['variable'].map(lookup)


,variable,spearman,mutual_info,mi_kde,abs_spearman,mi_kde_clipped,real_name
211,macro_11,0.809455,1.669447,0.654837,0.809455,0.654837,G_farms
208,macro_8,-0.809455,1.668637,0.654837,0.809455,0.654837,B_farms
209,macro_9,0.692573,0.869047,0.531185,0.692573,0.531185,GOS
210,macro_10,0.673368,0.838335,0.532264,0.673368,0.532264,GKS
206,macro_6,0.343648,0.039279,-21.852323,0.343648,0.000000,Total_P_G
200,macro_0,0.343520,0.040793,-33.843841,0.343520,0.000000,Total_G
204,macro_4,0.314327,0.040825,-33.843841,0.314327,0.000000,real_gamma
207,macro_7,-0.192469,0.054752,0.140042,0.192469,0.140042,Total_P_B
203,macro_3,-0.191297,0.055859,0.140184,0.191297,0.140184,real_beta
205,macro_5,-0.184747,0.055555,0.140042,0.184747,0.140042,GDP


In [41]:
print(macro_vars[['variable','real_name','spearman', 'mi_kde_clipped']].style.format(precision=3).hide(axis='index').to_latex()
        )

\begin{tabular}{llrr}
variable & real_name & spearman & mi_kde_clipped \\
macro_11 & G_farms & 0.809 & 0.655 \\
macro_8 & B_farms & -0.809 & 0.655 \\
macro_9 & GOS & 0.693 & 0.531 \\
macro_10 & GKS & 0.673 & 0.532 \\
macro_6 & Total_P_G & 0.344 & 0.000 \\
macro_0 & Total_G & 0.344 & 0.000 \\
macro_4 & real_gamma & 0.314 & 0.000 \\
macro_7 & Total_P_B & -0.192 & 0.140 \\
macro_3 & real_beta & -0.191 & 0.140 \\
macro_5 & GDP & -0.185 & 0.140 \\
macro_1 & Total_B & -0.178 & 0.140 \\
macro_2 & Total_K & -0.171 & 0.140 \\
\end{tabular}



In [ ]:
# from scipy.stats import gaussian_kde
# import numpy as np

# def kde_mi(x, y, gridsize=100, bw_method=None):
#     """
#     KDE-based mutual information matching ISOKANN.jl implementation
#     """
#     # Create grid
#     xg = np.linspace(x.min(), x.max(), gridsize)
#     yg = np.linspace(y.min(), y.max(), gridsize)
#     dx = xg[1] - xg[0]
#     dy = yg[1] - yg[0]
    
#     # Estimate joint density
#     xy = np.vstack([x, y])
#     kde_joint = gaussian_kde(xy, bw_method=bw_method)
#     xg_mesh, yg_mesh = np.meshgrid(xg, yg)
#     positions = np.vstack([xg_mesh.ravel(), yg_mesh.ravel()])
#     pxy = kde_joint(positions).reshape(gridsize, gridsize).T
    
#     # Estimate marginals
#     px = np.sum(pxy, axis=1, keepdims=True) * dy
#     py = np.sum(pxy, axis=0, keepdims=True) * dx
    
#     # Compute MI (avoid log(0))
#     px_py = px @ py
#     with np.errstate(divide='ignore', invalid='ignore'):
#         integrand = pxy * np.log(pxy / px_py)
#         integrand[~np.isfinite(integrand)] = 0
    
#     return np.sum(integrand) * dx * dy

# # Check what columns you actually have
# print("DataFrame columns:", df.columns.tolist())
# print("\nLabels list:", labels[:5])  # Show first 5 labels

# # Use the CORRECT column names from your DataFrame
# # These should match what you created earlier
# feature_columns = [col for col in df.columns if col != "chi"]

# # Compute KDE-based MI for each feature
# print("\nComputing KDE-based Mutual Information...")
# mi_kde = [kde_mi(df["chi"].values, df[col].values) for col in feature_columns]

# # Add to your results
# results_kde = pd.DataFrame({
#     "variable": feature_columns,
#     "mi_kde": mi_kde
# })

# print("\nTop 10 by KDE MI:")
# results_kde.sort_values("mi_kde", ascending=False, inplace=True)
# print(results_kde.head(10))

# # Optional: merge with your existing results
# # results = results.merge(results_kde, on="variable", how="left")

In [ ]:
#Save results so analysis don#t have to be done everytime
results.to_csv('results_correlation_validation.csv', index=False)

In [ ]:
# #read results
# results_kde = pd.read_csv('results_kde.csv')
# results_kde.reset_index(drop=True, inplace=True)